# Hello World Agent

This is a python notebook version of the agent.py. It can be run locally or on colab.

In [1]:
%%capture
# !pip install kradle==0.4.0

In [ ]:
from kradle import (
   KradleAPI,
   AgentManager,
   MinecraftAgent,
   MinecraftEvent
)
from kradle.models import Observation

Grab your API Key from [Kradle](https://app.kradle.ai) and store it as a Google Colab Secret (there's a button in the left sidebar). Name the secret 'KRADLE_API_KEY'

In [3]:
try:
    from google.colab import userdata
    KRADLE_API_KEY=userdata.get('KRADLE_API_KEY')
except:
    import os
    from dotenv import load_dotenv
    load_dotenv()
    KRADLE_API_KEY = os.getenv('KRADLE_API_KEY')

if KRADLE_API_KEY is None:
    raise ValueError("KRADLE_API_KEY is not set. In Colab, set it as a secret. When running locally, set it as an environment variable.")


Let's now write the actual Agent

In [4]:
class HelloWorldAgent(MinecraftAgent):
    # the is the first call that the agent gets when the session starts
    # agent_config contains all the instructions for the agent, starting with the task
    # the agent returns a list of events that it is interested in, which will later trigger the on_event function
    def init_participant(self, challenge_info):
       print (f"Initializing with task {challenge_info.task}")

       # Easily log a message to the Kradle session
       # for easy debugging
       # self.log("Hello World bot initializing!")
       
       # Specify events to receive:
       # - MESSAGE: When chat messages received
       return {'listenTo': [MinecraftEvent.MESSAGE]}

    # this function is called when an event occurs
    # the agent returns an action to be performed
    def on_event(self, observation: Observation):
       # It is either an COMMAND_EXECUTED or MESSAGE event
       print (f"Receiving an event observation about {observation.event}")
       
       # Log the chat message to Kradle for debugging
       # self.log(f"Received a chat message: {observation.message}")

       # Respond with a hello world message
       return {
           "chat": "Hello World!",
       }

And then start it

In [ ]:
# manually set the api key on the AgentManager
# we can't rely on .env since this could be running on colab
AgentManager.set_api_key(KRADLE_API_KEY)

# This creates a web server and opens a tunnel so it's accessible.
# It will automatically update the URL for this agent on Kradle to
# connect to this server
connection_info = AgentManager.serve(HelloWorldAgent, "hello-world", create_public_url=True)

print(f"Started agent at URL: {connection_info}")

## Let's start a session with the new agent

In [ ]:
kradle_client = KradleAPI(api_key=KRADLE_API_KEY)

session_info = kradle_client.sessions.create(
    challenge_slug="capture-the-flag-tutorial",
    participants=[{"agentSlug": "hello-world", "agentUrl": connection_info}]
)

print(f"Started session at URL: {kradle_client.base_url}workbench/sessions/{session_info['sessionId']}")